In [1]:
import pandas as pd
import numpy as np

In [46]:
class NJCleaner:
    def __init__(self,path="2018_03.csv") -> None:
        self.data=pd.read_csv(path)
    def order_by_scheduled_time(self)->pd.DataFrame:
        #newdata=self.data.copy()
        return self.data.sort_values(by=["scheduled_time"])
    def drop_columns_and_nan(self)->pd.DataFrame:
        colum=self.data.drop(columns=['from','to'])
        nan= colum.dropna()
        self.data=nan
        return self.data
    def convert_date_to_day(self)->pd.DataFrame:
        self.data['day']=pd.to_datetime(self.data['date']).dt.day_name()
        return self.data
    def convert_scheduled_time_to_part_of_the_day(self)->pd.DataFrame:
        self.data["part_of_the_day"]=pd.to_datetime(self.data['scheduled_time']).dt.hour.apply(lambda time:'early_morning' if time>=4 and time<8 
                                                                       else('morning'  if time>=8 and time<12  
                                                                       else ('afternoon' if time>=12 and time<16  
                                                                         else('evening' if time>=16 and time<20  
                                                                         else('night'if time>=20 and time<24  
                                                                         else 'late_night')))))
        
        colum=self.data.drop(columns=['scheduled_time'])
        self.data=colum
        return self.data   
    def convert_delay(self)->pd.DataFrame:
        self.data['delay']=self.data['delay_minutes'].apply(lambda delay:0 if delay<=5 else 1)
        return self.data
    def drop_unnecessary_columns(self)->pd.DataFrame:
        colum=self.data.drop(columns=['train_id','actual_time','delay_minutes'])
        self.data=colum
        return self.data
    def save_first_60k(self,path:str):
        sixtyk=self.data.iloc[:60000]
        sixtyk.to_csv(path,index=False)
    def prep_df(self,path:str):
        self.order_by_scheduled_time()
        self.drop_columns_and_nan()
        self.convert_date_to_day()
        self.convert_scheduled_time_to_part_of_the_day()
        self.convert_delay()
        self.drop_unnecessary_columns()
        self.save_first_60k(path)




        

pl=NJCleaner()
#pl.order_by_scheduled_time()
#pl.drop_columns_and_nan()
#pl.convert_date_to_day()
#pl.convert_scheduled_time_to_part_of_the_day()
#pl.convert_delay()
#pl.drop_unnecessary_columns()
pl.prep_df("nk.csv")




In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None) -> None:
        self.feature_index=feature_index
        self.threshold=threshold
        self.left=left
        self.right=right
        self.info_gain=info_gain
        self.value=value

class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        
        self.root = None
        
        self.min_samples_split = min_samples_split 
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            if best_split["info_gain"]>0: 
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                return Node(best_split["feature_index"], best_split["threshold"],
                            left_subtree, right_subtree, best_split["info_gain"])
        

        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        best_split = {}
        max_info_gain = -float("inf")
        
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            for threshold in possible_thresholds: 
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        
        if tree.value!=None: 
            return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

In [1]:
import pandas as pd
import numpy as np
path='2018_03.csv'
save_path='NJ_test.csv'

In [19]:
class NJCleaner:
    def __init__(self,path:str) -> None:
        self.data=pd.read_csv(path)
    def order_by_scheduled_time(self)->pd.DataFrame:
        #newdata=self.data.copy()
        return self.data.sort_values(by=["scheduled_time"])
    def drop_columns_and_nan(self)->pd.DataFrame:
       nan=self.data.dropna().drop(['from','to'],axis=1)
       self.data=nan
       return self.data
    def convert_date_to_day(self)->pd.DataFrame:
        self.data['day']=pd.to_datetime(self.data['date']).dt.day_name()
        drop=self.data.drop(['date'],axis=1)
        self.data=drop
        return self.data
    def convert_scheduled_time_to_part_of_the_day(self)->pd.DataFrame:
        self.data["part_of_the_day"]=pd.to_datetime(self.data['scheduled_time']).dt.hour.apply(lambda time:'early_morning' if time>=4 and time<8 
                                                                       else('morning'  if time>=8 and time<12  
                                                                       else ('afternoon' if time>=12 and time<16  
                                                                         else('evening' if time>=16 and time<20  
                                                                         else('night'if time>=20 and time<24  
                                                                         else 'late_night')))))
        
        colum=self.data.drop(columns=['scheduled_time'])
        self.data=colum
        return self.data   
    def convert_delay(self)->pd.DataFrame:
        self.data['delay']=self.data['delay_minutes'].apply(lambda delay:0 if delay<=5 else 1)
        return self.data
    def drop_unnecessary_columns(self)->pd.DataFrame:
        colum=self.data.drop(columns=['train_id','actual_time','delay_minutes'])
        self.data=colum
        return self.data
    def save_first_60k(self,save_path:str):
        sixtyk=self.data.iloc[:60000]
        self.data=sixtyk
        sixtyk.to_csv(save_path,index=False)
    def prep_df(self,save_path:str):
        self.order_by_scheduled_time()
        self.drop_columns_and_nan()
        self.convert_date_to_day()
        self.convert_scheduled_time_to_part_of_the_day()
        self.convert_delay()
        self.drop_unnecessary_columns()
        self.save_first_60k(save_path)
        self.data.to_csv(save_path)

testing=NJCleaner(path)
#testing.drop_columns_and_nan()
#testing.convert_date_to_day()
#testing.convert_scheduled_time_to_part_of_the_day()
#testing.convert_delay()
#testing.drop_unnecessary_columns()
testing.prep_df(save_path)